# Sampling methods

In [ ]:
%cd "~/lab/NB10422645"

from src.surrogate.sampling.util import plot2Ddemo, mpl3Ddemo, plotly3Dsurf, plot2Dline, plotFFTPeak, plotly3Dscatter
from src.problems.two_dimensional import *
import pandas as pd
test_geometry = MissionGeometry()
test_design = NOmegaPointsScaleBasedPeriodic(int(test_geometry.P_s / 10), 3)
test_spacecraft = Spacecraft()
test_problem = AttitudeTrajectoryProblem2D(test_geometry, test_spacecraft, test_design)

## Uninformed Sampling

### Grid Domain Sampling

In [ ]:
from src.surrogate.sampling.uninformed import grid_sample_domain

In [ ]:
# %%timeit
res1 = grid_sample_domain([[0,0,0],[1,1,1]],[10,10,10])
x, y, z = res1[:,0], res1[:,1], res1[:,2]
plotly3Dscatter(x,y,z, name='grid_domain_sampling')

### Random Domain Sampling

In [ ]:
from src.surrogate.sampling.uninformed import random_uniform_sample_domain

In [ ]:
# %%timeit
res2 = random_uniform_sample_domain([[0,0,0],[1,1,1]], int(1e3))
x, y, z = res2[:,0], res2[:,1], res2[:,2]
plotly3Dscatter(x,y,z, name='random_uniform_sample_domain')

### Halton Domain Sampling

In [ ]:
from src.surrogate.sampling.uninformed import halton_sample_domain

In [ ]:
# %%timeit
res3 = halton_sample_domain([[0,0,0],[1,1,1]], int(1e3))
x, y, z = res3[:,0], res3[:,1], res3[:,2]
plotly3Dscatter(x,y,z, name='halton_sample_domain')

### Multivariate Normal Point Sampling

In [ ]:
from src.surrogate.sampling.uninformed import multivariate_normal_sample_point

In [ ]:
point = [0.5, 0.5, 0.5]  # Mean
covar = 0.5e-1 * np.eye(3)
res4 = multivariate_normal_sample_point(point, covar, int(1e3))
x, y, z = res4[:,0], res4[:,1], res4[:,2]
plotly3Dscatter(x,y,z, name='multivariate_normal_point_sample')

## Informed Sampling

### Example Functions

In [6]:
%cd "~/lab/NB10422645"
from src.surrogate.sampling.util import plotly3Dsurf, plotly3Dtrisurf
from src.surrogate.sampling.uninformed import grid_sample_domain 
from src.surrogate.sampling.uninformed import random_uniform_sample_domain
from src.surrogate.sampling.uninformed import halton_sample_domain 
from src.surrogate.test import TestFunctionSet2DInputSpace

/home/ggarrett/lab/NB10422645


In [3]:
F2D = TestFunctionSet2DInputSpace()
print(F2D)

Available test functions in set: 
- Rastrigin2D
- Ackley
- Sphere2D
- Rosenbrock2D
- Beale
- GoldsteinPrice
- Booth
- BukinN6
- Matyas
- LeviN13
- Himmelblau
- ThreeHumpCamel
- Easom
- CrossInTray
- Eggholder
- HolderTable
- McCormick
- ScafferN2
- ScafferN4
- StyblinskiTang2D
- GarrettLagadec2D_pt6



In [4]:
def inspection_plot(F, N=100):
    f1_grid = grid_sample_domain(F.bounds,[N, N], flatten=False)
    x, y = f1_grid[0], f1_grid[1]
    plotly3Dsurf(x, y, F(x,y), static=False)

In [ ]:
# inspection_plot(F2D.Rastrigin2D)

In [ ]:
# inspection_plot(F2D.Ackley)

In [ ]:
# inspection_plot(F2D.Sphere2D)

In [ ]:
# inspection_plot(F2D.Beale)

In [ ]:
# inspection_plot(F2D.BukinN6)

In [ ]:
# inspection_plot(F2D.LeviN13)

In [ ]:
# inspection_plot(F2D.CrossInTray, N=92)

In [ ]:
# inspection_plot(F2D.HolderTable)

### Discrete Fourier Transform (DFT) - Fixed Grid Orientation

#### Demo Algorithm Landscape

In [7]:
%cd "~/lab/NB10422645"
from src.surrogate.sampling.util import plotly3Dsurf
from src.surrogate.sampling.informed import base_analysis
from src.surrogate.sampling.informed import dft_peaks
from src.surrogate.sampling.informed import *
from src.surrogate.sampling.uninformed import grid_sample_domain

/home/ggarrett/lab/NB10422645


Example use of ``dft_peaks`` algorithm.

In [8]:
dft_peaks_1(
    function      = Easom, 
    bounds        = Easom.bounds, 
    n_sample_dim  = [400, 400], 
    plot          = True,
    rel_threshold = None,
    abs_threshold = 0.01)

NameError: name 'Easom' is not defined

In [ ]:
# base_analysis(
#     function      = Easom, 
#     bounds        = Easom.bounds*0.1, 
#     N_init_mag    = 3, 
#     maxiter       = 300,
#     verbose       = False,
#     rel_threshold = None,
#     abs_threshold = 0.001)

In [9]:
def plot_for_threshold(function, bounds, abs_threshold=None, rel_threshold=None, show_sampled=True, name=None):
    freq, amplitudes = dft_peaks(function, bounds, n_sample_dim=[400,400], plot=False,
                                 abs_threshold=abs_threshold, 
                                 rel_threshold=rel_threshold)
    g = grid_sample_domain(bounds,[freq[0]*2+1, freq[1]*2+1], flatten=False, endpoint=True)
    x, y = g[0], g[1]
    plotly3Dtrisurf(x, y, function(x,y), show_scatter=show_sampled, name=name)
    print(freq)

def plot_f_from_grid(f, grid, samples=False):
    x, y = grid[0], grid[1]
    plotly3Dsurf(x, y, f(x, y), show_scatter=samples)


def plot_z_grids(x, y, z):
    plotly3Dsurf(x, y, z)

def bounds2vertices(bounds):
    return np.array(list(itertools.product(*np.array(bounds).T)))


def matmul_grid2D(e_v, e_grid):
    e_x_grid = np.zeros(e_grid.shape)
    e_x_grid[0] = e_v[0, 0] * e_grid[0] + e_v[0, 1] * e_grid[1]
    e_x_grid[1] = e_v[1, 0] * e_grid[0] + e_v[1, 1] * e_grid[1]
    return e_x_grid


def e_grid(bounds, N, e_v, flatten=False, endpoint=False):
    # Generate eigen space vertices from original bounds.
    e_vertices = np.matmul(np.linalg.inv(e_v), bounds2vertices(bounds).T).T
    e_bounds = np.array([np.min(e_vertices, axis=0), np.max(e_vertices, axis=0)])
    e_grid = grid_sample_domain(e_bounds,  # Generate meshgrid if init
                                N,
                                flatten=flatten,
                                endpoint=endpoint)
    return e_grid


def dft_peaks_1(function, bounds, n_sample_dim, 
                abs_threshold=None,
                rel_threshold=None,
                plot=False):
    
    # Pre-preparative coding tid bits.
    N = np.array(n_sample_dim)  # Ensure N is a numpy array.
    meshgrid = grid_sample_domain(bounds,  # Generate meshgrid if init
                                  N,
                                  flatten=False,
                                  endpoint=False)

    f = function(*meshgrid)
    f_range = np.max(f) - np.min(f)
    target = np.arange(len(f.shape))

    # Perform n dimensional DFT on target axes
    X = fftpack.fftn(f, axes=target)
    X_abs = np.abs(X)

    # Generate frequency spectrum for sample rate of each dimension.
    freqs = [fftpack.fftfreq(int(n)) * n for n in N]

    # Post preparative coding tid bits.
    a_pos_per_x = []  # Amplitude history per input dimension.
    f_pos_per_x = []  # Frequency history per input dimension.
    aux = np.arange(len(X_abs.shape))  # Index set {0, 1, ..., n_dim}
    
    for i in target:

        # Determine the one dimensional spectral analysis.
        axes_to_rid = tuple(aux[aux != i])
        axes_avg = np.mean(X_abs, axis=axes_to_rid)
        axes_avg[0] = 0  # Get rid of 0 Hz peak

        pos_bool = freqs[i] > 0
        neg_bool = freqs[i] < 0

        a_pos = 2 / (N[i]) * axes_avg[pos_bool]
        f_pos = freqs[i][pos_bool]
        a_pos_cum_reversed = np.maximum.accumulate(a_pos[::-1])[::-1]

        # Threshold present.
        if abs_threshold or rel_threshold:
            if abs_threshold and not rel_threshold:
                threshold = abs_threshold
            elif rel_threshold and not abs_threshold:
                threshold = f_range * rel_threshold
            elif abs_threshold and rel_threshold:
                threshold = np.min([abs_threshold, f_range * rel_threshold])
            try:
                a_pos_exceeded_x_i = a_pos[a_pos_cum_reversed <= threshold][0]
                f_pos_exceeded_x_i = f_pos[a_pos == a_pos_exceeded_x_i][0]
            except IndexError:
                a_pos_exceeded_x_i = a_pos[-1]
                f_pos_exceeded_x_i = f_pos[a_pos==a_pos[-1]][0]

        # Max amplitude convergence.
        else:
            raise Warning(
                f"""
            dft_peaks functionally required either:
                - relative threshold (rel_threshold)
                - absolute threshold (abs_threshold)

            But these arguments were found to be:
                - rel_threshold = {rel_threshold}
                - abs_threshold = {abs_threshold}

            The algorithm will converge to the maximum 
            deteced amplitude per x, by default.""")
            a_pos_exceeded_x_i = a_pos[a_pos == np.max(a_pos)]
            f_pos_exceeded_x_i = f_pos[a_pos == np.max(a_pos)]

        if plot:
            plotFFTPeak(f_pos, a_pos)
        
        f_pos_per_x.append(f_pos_exceeded_x_i)
        a_pos_per_x.append(a_pos_exceeded_x_i)
        
    return f_pos_per_x, a_pos_per_x, f_range
            


def dft_peaks2(function, bounds,
               n_sample_dim,
               rel_threshold=None,
               abs_threshold=None,
               dft_basis=None,
               plot=False,
               subplot=None):
    
    # Pre-preparative coding tid bits.
    N = np.array(n_sample_dim)  # Ensure N is a numpy array.
    
    if dft_basis is not None:
        meshgrid = e_grid(bounds,  # Generate meshgrid if init
                          N,
                          dft_basis,
                          flatten=False,
                          endpoint=False)
        meshgrid = matmul_grid2D(np.linalg.inv(dft_basis), meshgrid)

    else:
        meshgrid = grid_sample_domain(bounds,  # Generate meshgrid if init
                                      N,
                                      flatten=False,
                                      endpoint=False)

    f = function(*meshgrid)
    f_range = np.max(f) - np.min(f)
    target = np.arange(len(f.shape))

    # Perform n dimensional DFT on target axes
    X = fftpack.fftn(f, axes=target)
    X_abs = np.abs(X)

    X_1 = np.abs(fftpack.fftn(f, axes=(0)))
    X_2 = np.abs(fftpack.fftn(f, axes=(1)))

    # Generate frequency spectrum for sample rate of each dimension.
    freqs = [fftpack.fftfreq(int(n)) * n for n in N]

    # Post preparative coding tid bits.
    a_pos_per_x = []  # Amplitude history per input dimension.
    f_pos_per_x = []  # Frequency history per input dimension.
    a_pos_exceeded_per_x = []  # Amplitude history per input dimension.
    f_pos_exceeded_per_x = []  # Frequency history per input dimension.
    aux = np.arange(len(X_abs.shape))  # Index set {0, 1, ..., n_dim}

    #     X_abs = np.swapaxes(X_abs, 0, 1) if len(X_abs.shape) > 1 else X_abs
    # Swaps axes 0 and 1 (for numpy convention of dealing with arrays)
    # test_lam = lambda x: 0 if x is 1 else 1 if x is 0 else x

    #     print(X_abs)
    # def f__(freqs, N, X_abs):
    #     mg = np.meshgrid(*freqs)
    #     X_abs[mg[0] == 0] == 0
    #     X_abs[mg[1] == 0] == 0
    #     _x = mg[0][(mg[0] > 0) & (mg[1] > 0)]
    #     _y = mg[1][(mg[0] > 0) & (mg[1] > 0)]
    #     _z = X_abs[(mg[0] > 0) & (mg[1] > 0)] / N
    #     print(_x.shape)
    #     print(_y.shape)
    #     print(_z.shape)
    #     sq = int(np.sqrt(_z.shape[0]))
    #     plot_z_grids(_x.reshape(sq, sq), _y.reshape(sq, sq), _z.reshape(sq, sq))
    # 
    # f__(freqs, np.product(N), X_abs)
    # f__(freqs, 1, X_1)
    # f__(freqs, 1, X_2)

    for i in target:

        # Determine the one dimensional spectral analysis.
        axes_to_rid = tuple(aux[aux != i])
        axes_avg = np.mean(X_abs, axis=axes_to_rid)
        axes_avg[0] = 0  # Get rid of 0 Hz peak

        pos_bool = freqs[i] > 0
        neg_bool = freqs[i] < 0

        a_pos = 2 / (N[i]) * axes_avg[pos_bool]
        f_pos = freqs[i][pos_bool]
        a_pos_per_x.append(a_pos)
        f_pos_per_x.append(f_pos)

        # Threshold present.
        if abs_threshold or rel_threshold:
            if abs_threshold and not rel_threshold:
                threshold = abs_threshold
            elif rel_threshold and not abs_threshold:
                threshold = f_range * rel_threshold
            elif abs_threshold and rel_threshold:
                threshold = np.min([abs_threshold, f_range * rel_threshold])
            a_pos_exceeded_x_i = a_pos[a_pos >= threshold]
            f_pos_exceeded_x_i = f_pos[a_pos >= threshold]

        # Max amplitude convergence.
        else:
            raise Warning(
                f"""
            dft_peaks functionally required either:
                - relative threshold (rel_threshold)
                - absolute threshold (abs_threshold)

            But these arguments were found to be:
                - rel_threshold = {rel_threshold}
                - abs_threshold = {abs_threshold}

            The algorithm will converge to the maximum 
            deteced amplitude per x, by default.""")
            a_pos_exceeded_x_i = a_pos[a_pos == np.max(a_pos)]
            f_pos_exceeded_x_i = f_pos[a_pos == np.max(a_pos)]
        
#         if
        f_pos_exceeded_per_x.append(f_pos_exceeded_x_i)
        a_pos_exceeded_per_x.append(a_pos_exceeded_x_i)

        if plot:
            plotFFTPeak(f_pos, a_pos)
            
        if subplot is not None:
            subplot[i].stem(f_pos, a_pos)
            

#     available_f_throughout = np.intersect1d(*f_pos_per_x)
#     intersection_violated_f = np.intersect1d(*f_pos_exceeded_per_x)
#     all_violated_f = np.union1d(*f_pos_exceeded_per_x)

#     if set(all_violated_f).issubset(set(available_f_throughout)):
#         # Construct H matrix for SVD.
#         H = np.vstack([a_i[np.isin(f_i, all_violated_f)] for a_i, f_i in zip(a_pos_per_x, f_pos_per_x)])
#         A = H[:, np.argsort(np.max(H, axis=0))[::-1][:len(bounds[0])]]

#     else:
#         #         raise Warning(
#         #         f"""
#         #         All violated signal components are not available for all x.

#         #         The following frequencies are not common throughout:
#         #         {np.setdiff1d(all_violated_f, available_f_throughout)}
#         #         """)
#         H = np.vstack([a_i[np.isin(f_i, intersection_violated_f)] for a_i, f_i in zip(a_pos_per_x, f_pos_per_x)])
#         A = H[:, np.argsort(np.max(H, axis=0))[::-1][:len(bounds[0])]]

    f_pos_exceeded_per_x

    return f_pos_exceeded_per_x, a_pos_exceeded_per_x








# # test= lambda x,y:
# # test_bounds = 
# # test_bounds = (10*np.array(test_bounds)+10)/100 * 200

# # F = F2D.CrossInTray
# F = Test

# abs_tol = 0.01

# N = [100]*2
# fv0, av0 = dft_peaks2(
#             function      = F, 
#             bounds        = F.bounds, 
#             n_sample_dim  = N, 
#             plot          = True,
#             rel_threshold = None,
#             abs_threshold = abs_tol)

# Nx1 = np.round(np.max(fv0[0]))*2 + 1
# Nx2 = np.round(np.max(fv0[1]))*2 + 1

# fv0, av0 = dft_peaks2(
#             function      = F, 
#             bounds        = F.bounds, 
#             n_sample_dim  = [Nx1, Nx2], 
#             plot          = True,
#             rel_threshold = None,
#             abs_threshold = abs_tol)

# plot_f_from_grid(F, grid1)
# plot_f_from_grid(F, grid_sample_domain(F.bounds,  # Generate meshgrid if init
#                                        [Nx1, Nx2],
#                                        flatten=False,
#                                        endpoint=True))

In [185]:
    
def algorithm_1(function, bounds, rel_threshold = None, abs_threshold = None, confidence=1, convergence=5, N_max=100):
    n_x = len(bounds[0])
    N_converged = [False] * n_x
    N = [3] * n_x
    f_p_x_history = None
    a_p_x_history = None
    f_p_x = None
    a_p_x = None
    while np.any(np.array(N_converged)==False) and np.product(N)<=N_max:
        f_p_x, a_p_x, f_range = dft_peaks_1(
            function      = function, 
            bounds        = bounds, 
            n_sample_dim  = N, 
            plot          = False,
            rel_threshold = rel_threshold,
            abs_threshold = abs_threshold)
        f_p_x = np.array(f_p_x)
        a_p_x = np.array(a_p_x)
        f_p_x_history = np.concatenate((f_p_x_history, f_p_x.reshape(1,-1))) if f_p_x_history is not None else f_p_x.reshape(1,-1)
        a_p_x_history = np.concatenate((a_p_x_history, a_p_x.reshape(1,-1))) if a_p_x_history is not None else a_p_x.reshape(1,-1)

        if len(f_p_x_history) >= convergence:
            for x_i in range(n_x):
                cond = np.array([f_p_x_history[-1-con_i][x_i] == f_p_x_history[-2-con_i][x_i] for con_i in range(0, convergence-1)])
                variance = np.var(a_p_x_history[-convergence:, x_i]/f_p_x_history[-convergence:, x_i])
                sigma = variance ** (1/2.)
#                 print(f_p_x_history[-convergence:, x_i])
#                 print(variance)
#                 mean = np.mean(f_p_x_history[-convergence:,x_i])
                rel_threshold = rel_threshold if rel_threshold is not None else np.infty
                abs_threshold = abs_threshold if abs_threshold is not None else np.infty
                threshold = np.min([rel_threshold * f_range, abs_threshold])
                
                if np.abs(sigma) * confidence<=threshold:        
                    N_converged[x_i] = True

                else:
                    N_converged[x_i] = False
                    N[x_i] += 1

        else:
            for x_i in range(n_x):
                N[x_i] += 1
    return f_p_x-1, a_p_x



for f in [F2D[1]]:
    print(f.__str__())
#     print(f)
#     F = f
#     f_conv, a_conv = algorithm_1(F, F.bounds, rel_threshold=0.01, confidence=5, convergence=20, N_max=1e5)
#     print(int(np.product(N_)))

    f_p_x, a_p_x, f_range = dft_peaks_1(
        function      = f, 
        bounds        = f.bounds, 
        n_sample_dim  = np.array([500,500]), 
        plot          = False,
        rel_threshold = 0.01,
        abs_threshold = False)
    
    N_ = np.array(f_p_x) * 2 + 1
    
    print(np.product(N_))

    plot_f_from_grid(f, grid_sample_domain(f.bounds,  # Generate meshgrid if init
                                           [np.max([N_[0],3]), np.max([N_[1],3])],
                                           flatten=False,
                                           endpoint=True), samples=True)

Ackley function 2D
529.0


In [11]:
from src.dataset import *
from src.surrogate.sampling.uninformed import grid_sample_domain 
from src.surrogate.sampling.uninformed import random_uniform_sample_domain
from src.surrogate.sampling.uninformed import halton_sample_domain 


In [139]:
def make_test_function_datasets(function, r_test = 0.2, r_valid = 0.2, tag="v1", transform=None, rel_threshold=0.1):
    f_p_x, a_p_x, f_range = dft_peaks_1(
    function      = function, 
    bounds        = function.bounds, 
    n_sample_dim  = np.array([500,500]), 
    plot          = False,
    rel_threshold = rel_threshold,
    abs_threshold = False)
    N_ = (np.array(f_p_x) * 2 + 1) 
    train_sample_x = grid_sample_domain(function.bounds, [np.max([N_[0],3]), np.max([N_[1],3])],
                                        flatten=True,
                                        endpoint=True)
    # Training ###############################################
    n_train_samples = train_sample_x[0].size
    F_train = function(*train_sample_x.T)
    F_train = F_train.reshape(-1, 1)
    ds_train = DataSetFX(input=train_sample_x, output=F_train, transform=transform)
    # Testing ################################################
    n_test_samples = np.max([int(n_train_samples/0.6 * 0.2), 100])
    test_sample_x = random_uniform_sample_domain(function.bounds, n_test_samples, seed = 69)
    F_test = function(*test_sample_x.T)
    F_test = F_test.reshape(-1, 1)
    if transform:
        ds_test = DataSetFX(input=ds_train.transform.tx1.object.transform(test_sample_x), 
                            output=ds_train.transform.tf1.object.transform(F_test))
    else:
        ds_test = DataSetFX(input=test_sample_x, 
                            output=F_test)
    # Validation #############################################      
    n_validation_samples = np.max([int(n_train_samples/0.6 * 0.2), 100])
    validation_sample_x = random_uniform_sample_domain(function.bounds, n_validation_samples, seed = 69*2)
    F_validation = function(*validation_sample_x.T)
    F_validation = F_validation.reshape(-1, 1)
    if transform:
        ds_validation = DataSetFX(input=ds_train.transform.tx1.object.transform(validation_sample_x), 
                                  output=ds_train.transform.tf1.object.transform(F_validation))
    else:
        ds_validation = DataSetFX(input=validation_sample_x, 
                                  output=F_validation)
    return ds_train, ds_test, ds_validation

In [ ]:
from src.surrogate.deeplearning.core import Trainer2
from src.surrogate.deeplearning.util import EarlyStopping

from src.surrogate.modules import activation_dict
from src.dataset import *
from sklearn import preprocessing
from src.dataset import *


    
    
tf1 = preprocessing.MinMaxScaler(feature_range=(0,1))
tx1 = preprocessing.MinMaxScaler(feature_range=(-1,1))

from torch import nn 

transform = TransformDataFrame.from_ordered_dict(OrderedDict([
    ("tf1", (tf1, ["f1"])),
    ("tx1", (tx1, [f"x{i}" for i in range(1,3)]))
]
))


import torch
from torch import nn


# def train_model_for_function(model, ):

    
class BasicBlock(nn.Module):
    def __init__(self, input_size, output_size, activation="relu"):
        super().__init__()
        self.activation = activation_dict[activation]
        self.block = nn.Sequential(
            nn.Linear(input_size,output_size),
            self.activation,
            nn.BatchNorm1d(output_size)
        )

    def forward(self, x):
        return self.block(x)

F = F2D[1]

train_ds, test_ds, validation_ds = make_test_function_datasets(F, transform=transform, rel_threshold=0.001)

model = nn.Sequential(
    BasicBlock(2, 100),
    
    BasicBlock(100, 100),
    BasicBlock(100, 100),
    BasicBlock(100, 100),
    BasicBlock(100, 100),
    
    BasicBlock(100, 1),
)

print(len(train_ds))
print(len(test_ds))


trainer = Trainer2(PATH = f"models/v2/{F.__str__().lower().replace(' ','_')}",
                   training_dataset = train_ds,
                   testing_dataset = test_ds,
                   validation_dataset = validation_ds,
                   model = model,
                   optimizer = torch.optim.Adam,
                   criterion = torch.nn.L1Loss,
                   batch_size = 50,
                   num_epochs = 2000,
                   learning_rate = 1e-2)

trainer.train(verbose=50, early_stopping=EarlyStopping(10, False, delta=0.05))

1849
100


/home/ggarrett/lab/NB10422645/src/surrogate/deeplearning/core.py:355: FutureWarning:

Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.




Epoch [  1  /2000 ]  |  Train Loss:  0.651412  |  Test Loss:  0.419457  |  lr:  0.01000
EarlyStopping counter: 1 out of 10
Epoch [ 51  /2000 ]  |  Train Loss:  0.050156  |  Test Loss:  0.066873  |  lr:  0.01000
Epoch [ 101 /2000 ]  |  Train Loss:  0.047217  |  Test Loss:  0.065167  |  lr:  0.01000
Epoch [ 151 /2000 ]  |  Train Loss:  0.041462  |  Test Loss:  0.057677  |  lr:  0.01000
Epoch [ 201 /2000 ]  |  Train Loss:  0.039842  |  Test Loss:  0.052479  |  lr:  0.01000
Epoch [ 251 /2000 ]  |  Train Loss:  0.040499  |  Test Loss:  0.066086  |  lr:  0.01000
EarlyStopping counter: 1 out of 10
Epoch [ 301 /2000 ]  |  Train Loss:  0.041527  |  Test Loss:  0.045780  |  lr:  0.01000
Epoch [ 351 /2000 ]  |  Train Loss:  0.042687  |  Test Loss:  0.056596  |  lr:  0.01000
EarlyStopping counter: 1 out of 10
Epoch [ 401 /2000 ]  |  Train Loss:  0.040157  |  Test Loss:  0.040743  |  lr:  0.01000
Epoch [ 451 /2000 ]  |  Train Loss:  0.042815  |  Test Loss:  0.045008  |  lr:  0.01000
Epoch [ 501 /20

In [189]:

x_grid = grid_sample_domain(F2D[1].bounds,  # Generate meshgrid if init
                           [100,100],
                           flatten=True,
                           endpoint=True).T

if train_ds.transform is not None:
    with torch.no_grad():
        trainer._model.eval()
        plotly3Dtrisurf(*x_grid, 
                        train_ds.transform.tf1.object.inverse_transform(trainer._model(torch.tensor(train_ds.transform.tx1.object.transform(x_grid.T)
            ).float())))
else:
    with torch.no_grad():
        trainer._model.eval()
        plotly3Dtrisurf(*x_grid, trainer._model(torch.tensor(x_grid.T).float()))



In [15]:
def save_test_function_datasets(datasets):
    try:
        assert len(datasets) == 3
    except AssertionError:
        raise AssertionError("Dataset list must be length 3, (test, train , validation)")
    

make_dataset(F2D[13])

(<src.dataset._core.DataSetFX at 0x7f5bf45914e0>,
 <src.dataset._core.DataSetFX at 0x7f5bf466d1d0>)

In [114]:
import pygmo as pg


def eval(x):
    return trainer._model(torch.tensor(np.array([*x]).reshape(1,2)).float()).detach().numpy().flatten()

class pt_model_problem:
    def __init__(self, model, problem):
        self._eval = eval
        self._model = model
        self._model.eval()
        self._model.cpu()
        self._problem = problem
        
    def fitness(self, x):
#         x = torch.Variable(torch.from_numpy(x))
        return eval(x)
    
    def get_bounds(self):
        return tuple(self._problem.bounds)
    
# algo = pg.algorithm(pg.gaco(gen=100))
# algo.set_verbosity(1)
# # algo.set_verbosity(1)
# problem = pt_model_problem(trainer._model, F2D[0])
# print(problem._problem.optimums)
# prob = pg.problem(problem)
# pop = pg.population(prob, 160)
# pop = algo.evolve(pop)

# print("\n")
# sol_ = (list(np.round(pop.champion_x, 2)), np.round(pop.champion_f, 2)[0])

print(sol_)

print(F2D[0](*sol_[0]))

0.01983271571728551


In [ ]:
# print(fv0)

# # print(S)
# # print(U)

# # print([2*np.max(fv0[0])+1, 2*np.max(fv0[0])+1])

H, A, fv1 = dft_peaks2(
        function      = F, 
        bounds        = F.bounds, 
        n_sample_dim  = [2*np.max(fv0[0])+1, 2*np.max(fv0[1])+1], 
        plot          = True,
        rel_threshold = None,
        abs_threshold = abs_tol,
        dft_basis =  None)

e_m, e_v = np.linalg.eig(A)
# print(e_m)
U, S, VH = svd(H)


# print(fv1)


f_e = np.array([np.max(fv0[1]), np.max(fv0[0])])
# f_e = (np.abs(f_e)+1).astype(int)
# print(f_e)

# print(f_e)

H, A, fv = dft_peaks2(
        function      = F, 
        bounds        = F.bounds, 
        n_sample_dim  = [2*f_e[0]+1, 2*f_e[1]+1], 
        plot          = True,
        rel_threshold = None,
        abs_threshold = abs_tol,
        dft_basis = e_v)


grid_init = grid_sample_domain(F.bounds, # Generate meshgrid if init
                                [2*np.max(fv0[1])+1, 2*np.max(fv0[0])+1], 
                                flatten=False, 
                                endpoint=True)

grid_e = e_grid(F.bounds, # Generate meshgrid if init
                  [2*f_e[1]+1, 2*f_e[0]+1], 
                  e_v,
                  flatten=False, 
                  endpoint=True)

grid_e_x = matmul_grid2D(np.linalg.inv(e_v), grid_e)

grid_init = grid_sample_domain(F.bounds, # Generate meshgrid if init
                                N
                                , 
                                flatten=False, 
                                endpoint=True)

plot_f_from_grid(F, grid_init)
plot_f_from_grid(F, grid_e_x)


# grid_init = grid_sample_domain(BukinN6.bounds, # Generate meshgrid if init
#                                 [10,10], 
#                                 flatten=False, 
#                                 endpoint=True)


# grid_e = e_grid(BukinN6.bounds, # Generate meshgrid if init
#                   [10,10], 
#                   e_v,
#                   flatten=False, 
#                   endpoint=True)


# grid_init = grid_sample_domain(BukinN6.bounds, # Generate meshgrid if init
#                                 [10,10], 
#                                 flatten=False, 
#                                 endpoint=True)

# grid_e_x = matmul_grid(grid_e, e_v)





# grid_e = e_grid(BukinN6.bounds, # Generate meshgrid if init
#                   [10,10], 
#                   e_v,
#                   flatten=False, 
#                   endpoint=True)


# grid_init = grid_sample_domain(BukinN6.bounds, # Generate meshgrid if init
#                                 [10,10], 
#                                 flatten=False, 
#                                 endpoint=True)

# grid_e_x = matmul_grid(grid_e, e_v)


# plt.figure(figsize=(4,4), dpi=200)
# plt.gca().set_aspect('equal')

# plt.scatter(grid_init[0].flatten(), grid_init[1].flatten())
# plt.scatter(grid_e_x[0].flatten(), grid_e_x[1].flatten())

In [ ]:


# print(test_bounds[1]*np.eye(2))
H = dft_peaks2(
    function      = test, 
    bounds        = test_bounds, 
    n_sample_dim  = [200, 200], 
    plot          = False,
    rel_threshold = None,
    abs_threshold = 0.01)


# print(u)
# print(s)
# print(vh)

# def rotate_grid(grid, u, bounds):
#     center = np.mean(bounds, axis=0)
#     aux = np.zeros(grid.shape)
#     for idx, dim in enumerate(grid):
#         aux[idx] = dim - center[idx]
#     for idx, dim in enumerate(aux):
#         aux[idx] = np.sum([u[idx, j] * dim for j in 
        
# #     x1 = X[0]
# #     x2 = X[1]
# #     x1 = x1 - x1_c
# #     x2 = x2 - x2_c
# #     x_ = u[0,0] * x1 + u[0,1] * x2
# #     y_ = u[1,0] * x1 + u[1,1] * x2
#     return np.array([x_+x1_c,y_+x2_c])

# def test_me(u, bounds,



# f1_grid = rotate_grid(f1_grid, U[1])
            
# x, y = f1_grid[0], f1_grid[1]

# plotly3Dsurf(x, y, BukinN6(x,y))

# f1_grid = rotate_grid(f1_grid, U[2])
            
# x, y = f1_grid[0], f1_grid[1]

# plotly3Dsurf(x, y, BukinN6(x,y))

In [ ]:
F = LeviN13

rel = 0.2

abs_ = rel * 2000


dft_peaks(
    func          = F, 
    bounds        = F.bounds, 
    n_sample_dim  = [400, 400], 
    plot          = True,
    rel_threshold = None,
    abs_threshold = abs_)

# base_analysis(
#     function      = F, 
#     bounds        = F.bounds, 
#     N_init_mag    = 3, 
#     maxiter       = 200,
#     verbose       = False,
#     rel_threshold = None,
#     abs_threshold = abs_)

# plot_for_threshold(
#     function      = F, 
#     bounds        = F.bounds, 
#     rel_threshold = None,
#     abs_threshold = abs_
# )

In [ ]:
f1_grid = grid_sample_domain(F.bounds,[300,300])
x, y = f1_grid[:,0], f1_grid[:,1]
plotly3Dtrisurf(x, y, F(x,y), static=False, show_scatter=False, name="Lévi function N.13")


In [ ]:
plot_for_threshold(
    function      = LeviN13, 
    bounds        = LeviN13.bounds, 
    rel_threshold = None,
    abs_threshold = 30,
    show_sampled=False,
    name          = "$DFT\;Informed\;Grid\;Sampling\;(N_x=5,N_y=123,N_s=5\cdot{}123=615)$"
)

In [ ]:
N = (2*2+1)*(2*61+1)
F = LeviN13

f1_grid = halton_sample_domain(F.bounds,N)
x, y = f1_grid[:,0], f1_grid[:,1]
plotly3Dtrisurf(x, y, F(x,y), static=False, show_scatter=True, 
                name="$Uninformed\;Halton\;Sampling\;(N_s=615)$")

f1_grid = random_uniform_sample_domain(F.bounds,N)
x, y = f1_grid[:,0], f1_grid[:,1]
plotly3Dtrisurf(x, y, F(x,y), static=False, show_scatter=True, 
                name="$Random\;Uniform\;Sampling\;(N_s=615)$")

In [ ]:
# 2D Problem Dataset Analysis
solution = np.array([0.00192016,
                    -0.00135844,
                     0.00226742,
                     0.40646950,
                     0.28561942,
                     0.96828936,
                     0.50069615])

idx_ana = np.array([0,1])
f_idx = 0

def fun_test_1(*args, f_idx=f_idx):
    for i, _idx in enumerate(idx_ana):
        solution[_idx] = args[i]
    return test_problem.fitness(solution)[f_idx]

vect = np.vectorize(fun_test_1)
bounds_ = test_problem.get_bounds()
bounds_ = np.array(bounds_)[:,idx_ana]*1.0

# dft_peaks(vect, bounds_, n_sample_dim=[50,50], plot=True)

In [ ]:
g = grid_sample_domain(bounds_,[100, 100], flatten=False)
x, y = g[0], g[1]
plotly3Dsurf(x, y, vect(x,y), show_scatter=False)

In [ ]:
plot_for_threshold(
    function      = vect, 
    bounds        = bounds_, 
    rel_threshold = None,
    abs_threshold = 0.25,
    show_sampled=True
)

In [ ]:
g = grid_sample_domain(bounds_,[100, 100], flatten=False)
x, y = g[0], g[1]
plotly3Dsurf(x, y, vect(x,y), show_scatter=False)

In [ ]:
def plot_for_threshold(function, bounds, abs_threshold=None, rel_threshold=None):
    
    freq, amplitudes = dft_peaks(function, bounds, n_sample_dim=[50,50], plot=False,
                                abs_threshold=abs_threshold, 
                                 rel_threshold=rel_threshold)
    print(freq)
    g = grid_sample_domain(bounds*0.5,[freq[0]*2+1, freq[1]*2+1], flatten=False)
    
    x, y = g[0], g[1]
    
    plotly3Dsurf(x, y, function(x,y), show_scatter=True)

In [ ]:
plot_for_threshold(vect, bounds_, abs_threshold=0.2)

In [ ]:
base_analysis(vect, bounds_, N_init_mag=3, maxiter=50, verbose=True, abs_threshold=0.1)

In [ ]:
dft_peaks(f_rastrigin, bounds_rastrigin(2), n_sample_dim=[50,50], plot=True)
base_analysis(f_rastrigin, bounds_rastrigin(2), N_init_mag=3, maxiter=50, verbose=True, abs_threshold=0.5)

In [ ]:
base_analysis(f_himmelblau, bounds_himmelblau, N_init_mag=3, maxiter=50, verbose=True, rel_threshold=0.1)

In [ ]:
# Save random dataset.

sample_random_data = random_uniform_sample_domain(test_problem.get_bounds(), 6**7)
result_random_data = process_samples(sample_random_data, test_problem)
df_random_sampling = pd.DataFrame(columns=[f"f{i+1}" for i in range(test_problem.fitness_dim)] + [f"x{i+1}" for i in range(len(test_problem.design_space.get_bounds()[0]))], data=result_random_data)
df_random_sampling.to_parquet(f"data/AttitudeTrajectoryProblem2D_C1_R1/dataSet2_randomSampled_6.parquet")

In [ ]:
# Save halton dataset

sample_halton_data = halton_sample_domain(test_problem.get_bounds(), 6**7)
result_halton_data = process_samples(sample_halton_data, test_problem)
df_halton_sampling = pd.DataFrame(columns=[f"f{i+1}" for i in range(test_problem.fitness_dim)] + [f"x{i+1}" for i in range(len(test_problem.design_space.get_bounds()[0]))], data=result_halton_data)
df_halton_sampling.to_parquet(f"data/AttitudeTrajectoryProblem2D_C1_R1/dataSet2_haltonSampled_6.parquet")

In [ ]:
sample_informed_data = grid_sample_domain(test_problem.get_bounds(), [16, 16, 16, 3, 3, 3, 3])
result_informed_data = process_samples(sample_informed_data, test_problem)
df_informed_sampling = pd.DataFrame(columns=[f"f{i+1}" for i in range(test_problem.fitness_dim)] + [f"x{i+1}" for i in range(len(test_problem.design_space.get_bounds()[0]))], data=result_informed_data)
df_informed_sampling.to_parquet(f"data/AttitudeTrajectoryProblem2D_C1_R1/dataSet2_fftSampled_16_3_3e5.parquet")

sample_informed_data = grid_sample_domain(test_problem.get_bounds(), [16, 16, 16, 4, 4, 4, 4])
result_informed_data = process_samples(sample_informed_data, test_problem)
df_informed_sampling = pd.DataFrame(columns=[f"f{i+1}" for i in range(test_problem.fitness_dim)] + [f"x{i+1}" for i in range(len(test_problem.design_space.get_bounds()[0]))], data=result_informed_data)
df_informed_sampling.to_parquet(f"data/AttitudeTrajectoryProblem2D_C1_R1/dataSet2_fftSampled_16_4_1e6.parquet")

sample_informed_data = grid_sample_domain(test_problem.get_bounds(), [15, 15, 15, 4, 4, 4, 4])
result_informed_data = process_samples(sample_informed_data, test_problem)
df_informed_sampling = pd.DataFrame(columns=[f"f{i+1}" for i in range(test_problem.fitness_dim)] + [f"x{i+1}" for i in range(len(test_problem.design_space.get_bounds()[0]))], data=result_informed_data)
df_informed_sampling.to_parquet(f"data/AttitudeTrajectoryProblem2D_C1_R1/dataSet2_fftSampled_15_4_9e5.parquet")

sample_informed_data = grid_sample_domain(test_problem.get_bounds(), [14, 14, 14, 4, 4, 4, 4])
result_informed_data = process_samples(sample_informed_data, test_problem)
df_informed_sampling = pd.DataFrame(columns=[f"f{i+1}" for i in range(test_problem.fitness_dim)] + [f"x{i+1}" for i in range(len(test_problem.design_space.get_bounds()[0]))], data=result_informed_data)
df_informed_sampling.to_parquet(f"data/AttitudeTrajectoryProblem2D_C1_R1/dataSet2_fftSampled_14_4_7e5.parquet")

sample_informed_data = grid_sample_domain(test_problem.get_bounds(), [13, 13, 13, 4, 4, 4, 4])
result_informed_data = process_samples(sample_informed_data, test_problem)
df_informed_sampling = pd.DataFrame(columns=[f"f{i+1}" for i in range(test_problem.fitness_dim)] + [f"x{i+1}" for i in range(len(test_problem.design_space.get_bounds()[0]))], data=result_informed_data)
df_informed_sampling.to_parquet(f"data/AttitudeTrajectoryProblem2D_C1_R1/dataSet2_fftSampled_13_4_6e5.parquet")

In [ ]:
sample_informed_data = grid_sample_domain(test_problem.get_bounds(), [17, 17, 17, 3, 3, 3, 3])
result_informed_data = process_samples(sample_informed_data, test_problem)
df_informed_sampling = pd.DataFrame(columns=[f"f{i+1}" for i in range(test_problem.fitness_dim)] + [f"x{i+1}" for i in range(len(test_problem.design_space.get_bounds()[0]))], data=result_informed_data)
df_informed_sampling.to_parquet(f"data/AttitudeTrajectoryProblem2D_C1_R1/dataSet2_fftSampled_17_3_4e5.parquet")

sample_informed_data = grid_sample_domain(test_problem.get_bounds(), [18, 18, 18, 3, 3, 3, 3])
result_informed_data = process_samples(sample_informed_data, test_problem)
df_informed_sampling = pd.DataFrame(columns=[f"f{i+1}" for i in range(test_problem.fitness_dim)] + [f"x{i+1}" for i in range(len(test_problem.design_space.get_bounds()[0]))], data=result_informed_data)
df_informed_sampling.to_parquet(f"data/AttitudeTrajectoryProblem2D_C1_R1/dataSet2_fftSampled_18_3_5e5.parquet")

sample_informed_data = grid_sample_domain(test_problem.get_bounds(), [19, 19, 19, 3, 3, 3, 3])
result_informed_data = process_samples(sample_informed_data, test_problem)
df_informed_sampling = pd.DataFrame(columns=[f"f{i+1}" for i in range(test_problem.fitness_dim)] + [f"x{i+1}" for i in range(len(test_problem.design_space.get_bounds()[0]))], data=result_informed_data)
df_informed_sampling.to_parquet(f"data/AttitudeTrajectoryProblem2D_C1_R1/dataSet2_fftSampled_19_3_5e6.parquet")

sample_informed_data = grid_sample_domain(test_problem.get_bounds(), [20, 20, 20, 3, 3, 3, 3])
result_informed_data = process_samples(sample_informed_data, test_problem)
df_informed_sampling = pd.DataFrame(columns=[f"f{i+1}" for i in range(test_problem.fitness_dim)] + [f"x{i+1}" for i in range(len(test_problem.design_space.get_bounds()[0]))], data=result_informed_data)
df_informed_sampling.to_parquet(f"data/AttitudeTrajectoryProblem2D_C1_R1/dataSet2_fftSampled_20_3_6e6.parquet")

sample_informed_data = grid_sample_domain(test_problem.get_bounds(), [21, 21, 21, 3, 3, 3, 3])
result_informed_data = process_samples(sample_informed_data, test_problem)
df_informed_sampling = pd.DataFrame(columns=[f"f{i+1}" for i in range(test_problem.fitness_dim)] + [f"x{i+1}" for i in range(len(test_problem.design_space.get_bounds()[0]))], data=result_informed_data)
df_informed_sampling.to_parquet(f"data/AttitudeTrajectoryProblem2D_C1_R1/dataSet2_fftSampled_21_3_8e6.parquet")

In [ ]:

def grid_sample_problem(problem, n=10, scale_bounds=1.0, fixed=(())):
    
    bounds = scale_bounds * np.array(problem.design_space.get_bounds())    
    grid = sampled_grid(bounds, n, flatten=True)
    grid = pd.DataFrame(grid.T, columns=[f"x{i+1}" for i in range(len(grid.T[0]))])
    grid_ = grid
    grid_values = grid_.values
    perc_val = int(0.1 * len(grid_values))
    
    # Generate storage matrix.
    res=np.zeros((grid_.values.shape[0], 12))   # Add f,x shape attribute to problem.
    count=0
    for idx,_x in enumerate(grid_values):
        count+=1
        f = problem.fitness(_x)
        res[idx] = np.concatenate((f,_x))
        if (count % perc_val) == 0:
            print(round(count/len(grid_values) * 100, 1), r"%")
        
    df = pd.DataFrame(columns=[f"f{i+1}" for i in range(problem.fitness_dim)] + [f"x{i+1}" for i in range(len(test_problem.design_space.get_bounds()[0]))], data=res)
#     if os.path.exists("data"):
#         pass
#     else:
#         os.mkdir("data")
        
    save_name = f"data/AttitudeTrajectoryProblem2D_C1_R1/dataSet2_gridSampled_{n}.parquet"
    
    df.to_parquet(save_name)


In [ ]:
for n in np.arange(8,12):
    grid_sample_problem(test_problem, n)
    

## Case 2: Grid Sampled Optimal Region

In [ ]:
def grid_sample_solution(problem, solution, n, scale_per_bound=0.025):
#     print(x)
    
    # Get problem bounds.
    bounds = np.array(problem.design_space.get_bounds())
    
    # Transform into bound range.
    bound_range = bounds[1]-bounds[0]
    
#     print(bound_range)
    
    # Scale bound range for about point basis.
    scaled_bound_range = scale_per_bound * bound_range

    # Create new bounds centralised about point.
    bounds = np.concatenate(([solution+scaled_bound_range],[solution-scaled_bound_range]))
    
#     print(bounds.shape)
    
    # Sample bounds about point.
    grid = sampled_grid(bounds, n, flatten=True)
    
#     print(grid.shape)
    
    # Transform to dataframe.
    grid = pd.DataFrame(grid.T, columns=[f"x{i+1}" for i in range(len(grid.T[0]))])
    grid_ = grid
    grid_values = grid_.values
    perc_val = int(0.1 * len(grid_values))
    
    # Generate storage matrix.
    res=np.zeros((grid_.values.shape[0], 12))   # Add f,x shape attribute to problem.
    count=0
    for idx,_x in enumerate(grid_values):
        count+=1
#         print(_x)
        f = problem.fitness(_x)
        res[idx] = np.concatenate((f,_x))
        if (count % perc_val) == 0:
            print(round(count/len(grid_values) * 100, 1), r"%")
        
    df = pd.DataFrame(columns=[f"f{i+1}" for i in range(problem.fitness_dim)] + [f"x{i+1}" for i in range(len(test_problem.design_space.get_bounds()[0]))], data=res)
        
    save_name = f"data/AttitudeTrajectoryProblem2D_C1_R1/dataSet2_sol1_gridSampled_{n}.parquet"
    
    df.to_parquet(save_name)


x = np.array([0.00192016,
             -0.00135844,
              0.00226742,
              0.4064695,
              0.28561942,
              0.96828936,
              0.50069615])

x[-4:] = x[-4:] / np.linalg.norm(x[-4:])

In [ ]:
for n in np.arange(3,12):
    grid_sample_solution(test_problem, x, n, scale_per_bound=0.025)
    

